In [1]:
collection = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

In [2]:
from collections import defaultdict

# DTM
documents = defaultdict(lambda: defaultdict(int))
vocabulary = list()

for i, d in enumerate(collection): # i : 문서제목, d : i번째 문서 내 단어 목록
    for term in d:
        documents[i][term.lower()] += 1 # 아직 Z assignment는 안한상태. 
        vocabulary.append(term.lower())
        
vocabulary = list(set(vocabulary))

In [3]:
alpha = 0.1 
beta = 0.1

K = 3 # 토픽 개수

M = len(documents)
V = len(vocabulary)

# 특정 토픽에 몇 개의 단어가 있는지 => 분모
topicTermCount = defaultdict(int)

# 특정 문서의 단어에 상관없이 토픽 할당 횟수
docTopicDistribution = defaultdict(lambda: defaultdict(int))
# [document][0번째토픽:몇개의단어, 1번째토픽:몇개의단어]정보를 담고있음. 

#문서에 상관없이 특정 단어의 토픽 할당 횟수
topicTermDistribution = defaultdict(lambda: defaultdict(int))
#[topic][vocabulary 0:몇번, ..., n]

# z_ml = m번째 문서 1번째 단어의 Topic
# M개의 문서만큼 => N개의 단어 => Topic
termTopicAssignmentMatrix = defaultdict(lambda: defaultdict(int))
# Z[document][term] = Topic
# n(i, (j, r)) = i번째 토픽의 횟수, j번째 문서의 r번째 단어

In [4]:
# 기본적인 아이디어는 아래와 같다. 

# Z를 랜덤하게 initialize
from random import randrange, seed

seed(0) # 모두 같은 결과를 유도하기 위함(테스트)

for i, termList in enumerate(collection): # i번째 문서를 뽑음
    for j, term in enumerate(termList):
        token = term.lower()
        topic = randrange(K) # 0~2 사이의 int 하나를 받음. KNN과 같이 비지도학습이므로 컴퓨터가 알아서 토픽을 할당해 클러스터링해야한다. 
        
        # 총 100개의 단어가 나왔다면 아래 셋의 합은 100이 되어야 함. 
        topicTermCount[topic] += 1 # 2번식의 분모에 필요. (3짜리 list에 넣어줌)
        docTopicDistribution[i][topic] += 1 # 1번식때문에 필요(어느 doc의 몇번째 topic에 몇개만큼 단어가 할당되어 있는지)
        topicTermDistribution[topic][term] += 1 
        
        termTopicAssignmentMatrix[i][j] = topic # i번째 문서에서 나온 n(여기선 j)번째 단어에 대한 topic assignment. 즉 수식의 Z와 같음. (어느 문서에 나온 특정 위치를 골라내기 위해 사용.)

In [5]:
topicTermDistribution

defaultdict(<function __main__.<lambda>()>,
            {1: defaultdict(int,
                         {'Hadoop': 1,
                          'Big Data': 2,
                          'Java': 2,
                          'Storm': 1,
                          'Cassandra': 2,
                          'NoSQL': 1,
                          'MongoDB': 1,
                          'scipy': 1,
                          'R': 2,
                          'machine learning': 1,
                          'programming languages': 1,
                          'statistics': 1,
                          'probability': 2,
                          'Mahout': 1,
                          'neural networks': 1,
                          'deep learning': 1,
                          'artificial intelligence': 1,
                          'Python': 1}),
             0: defaultdict(int,
                         {'HBase': 2,
                          'Postgres': 2,
                          'scikit-learn': 1,

In [6]:
for i, topics in docTopicDistribution.items():
    print("{0:2d}번째 문서: {1}번 토픽".format(i, sorted(topics.items(), key = lambda x:x[1], reversed)))

SyntaxError: positional argument follows keyword argument (<ipython-input-6-61e208fb0cd1>, line 2)

In [7]:
topicTermCount

defaultdict(int, {1: 23, 0: 20, 2: 24})

In [8]:
def likelihoodAlpha(i, k):
    return docTopicDistribution[i][k] + alpha

In [9]:
def likelihoodBeta(k, term):
    return (topicTermDistribution[k][term] + beta) / (topicTermCount[k] + (beta *V))

In [10]:
from random import random

def collapsedGibbsSampling(i, term):
    sampling = list()
    for k in range(K):
        # k번째 토픽에 대한 확률
        sampling.append(likelihoodAlpha(i, k) * likelihoodBeta(k, term))
        
    threshold = sum(sampling) * random()    # 0~1 사이의 값 (sample 들 중에서 위치 선택) => 적당한 k번째를 고르기 위한 샘플링
    
    for topicNo, topicProbability in enumerate(sampling):
        threshold -= topicProbability
        
        if threshold <= 0.0:
            return topicNo
    

In [11]:
# 1000번 이터레이션하는부분 추가

iterationNumber = 1000

for _ in range(iterationNumber):
    for i, termList in enumerate(collection): # i번째 문서를 뽑음
        for j, term in enumerate(termList):
            topic = termTopicAssignmentMatrix[i][j]
            
            topicTermCount[topic] -= 1
            docTopicDistribution[i][topic] -= 1
            topicTermDistribution[topic][term] -= 1
            
            topic = collapsedGibbsSampling(i, term)
            
            
            # 총 100개의 단어가 나왔다면 아래 셋의 합은 100이 되어야 함. 
            topicTermCount[topic] += 1 # 2번식의 분모에 필요. (3짜리 list에 넣어줌)
            docTopicDistribution[i][topic] += 1 # 1번식때문에 필요(어느 doc의 몇번째 topic에 몇개만큼 단어가 할당되어 있는지)
            topicTermDistribution[topic][term] += 1 

            termTopicAssignmentMatrix[i][j] = topic # i번째 문서에서 나온 n(여기선 j)번째 단어에 대한 topic assignment. 즉 수식의 Z와 같음. (어느 문서에 나온 특정 위치를 골라내기 위해 사용.)

In [12]:
docTopicDistribution

defaultdict(<function __main__.<lambda>()>,
            {0: defaultdict(int, {1: 7, 0: 0, 2: 0}),
             1: defaultdict(int, {1: 5, 2: 0, 0: 0}),
             2: defaultdict(int, {2: 3, 0: 3, 1: 0}),
             3: defaultdict(int, {1: 0, 2: 3, 0: 2}),
             4: defaultdict(int, {1: 0, 0: 4, 2: 0}),
             5: defaultdict(int, {2: 3, 1: 1, 0: 2}),
             6: defaultdict(int, {1: 0, 2: 4, 0: 0}),
             7: defaultdict(int, {0: 4, 2: 0, 1: 0}),
             8: defaultdict(int, {2: 0, 1: 1, 0: 3}),
             9: defaultdict(int, {0: 0, 2: 0, 1: 4}),
             10: defaultdict(int, {2: 3, 0: 0, 1: 0}),
             11: defaultdict(int, {0: 3, 2: 1, 1: 0}),
             12: defaultdict(int, {0: 0, 2: 3, 1: 0}),
             13: defaultdict(int, {2: 0, 0: 0, 1: 5}),
             14: defaultdict(int, {0: 3, 2: 0, 1: 0})})

In [13]:
topicTermDistribution

defaultdict(<function __main__.<lambda>()>,
            {1: defaultdict(int,
                         {'Hadoop': 2,
                          'Big Data': 3,
                          'Java': 3,
                          'Storm': 1,
                          'Cassandra': 2,
                          'NoSQL': 1,
                          'MongoDB': 2,
                          'scipy': 0,
                          'R': 0,
                          'machine learning': 0,
                          'programming languages': 0,
                          'statistics': 0,
                          'probability': 0,
                          'Mahout': 0,
                          'neural networks': 0,
                          'deep learning': 0,
                          'artificial intelligence': 0,
                          'Python': 0,
                          'HBase': 3,
                          'Spark': 1,
                          'Postgres': 2,
                          'scikit-learn':